# Psyche-R1: Baseline vs LoRA Fine-Tuned (CPU-safe)

**Goal.** Show a simple, reproducible baseline vs fine-tuned comparison on a small mental-health counseling dataset (CounselChat), using a tiny instruct LLM (**Qwen2.5-0.5B-Instruct**) and **LoRA adapters** trained on **CPU**.

**What we’ll do**
1) Load & clean `counselchat-data.csv`
2) Build 80/10/10 train/dev/test splits
3) Baseline: zero-shot generation (no training)
4) LoRA fine-tune (CPU manual loop, small steps)
5) Evaluate: generate with the adapter, compare side-by-side
6) Export CSVs + quick empathy probe

> Note: CPU training is slow and minimal. Before start — Clone the repository and follow simple instructions from my github which the link can be find in 1st page of my report.


## Imports, Reproducibility, and Project Paths

This cell sets up the **runtime environment** for the notebook.

### What it does
- **Core libraries:**  
  - `os`, `pathlib.Path` for filesystem ops  
  - `math`, `json`, `re`, `gc` for utilities and cleanup  
  - `random`, `numpy`, `pandas` for data handling and reproducibility  
  - `sklearn.model_selection` for potential future splits (explicit split later)  
- **PyTorch & HF stack:**  
  - `torch` for tensors and training  
  - `datasets.Dataset` for wrapping Python lists → HF datasets  
  - `transformers` for tokenizer/model + training utilities  
  - `peft` for LoRA adapters (parameter-efficient fine-tuning)

### Reproducibility
We **fix the random seed** across Python, NumPy, and PyTorch:
```text
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)


In [1]:
import os, math, json, re, random, gc
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
                          DataCollatorForLanguageModeling, TrainingArguments, Trainer)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DATA_CSV_PATH = "../data/raw/counselchat-data.csv"
OUT_DIR = Path("runs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

c:\Users\User\miniconda3\envs\psyche-r1\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\User\miniconda3\envs\psyche-r1\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\User\miniconda3\envs\psyche-r1\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the g

## **Dataset Loading & Preprocessing**

This section loads the **CounselChat** dataset and prepares it for fine-tuning.  
Several important preprocessing steps are applied:

### **✔ HTML Cleaning**
CounselChat answers contain `<br>`, `<p>`, and other tags.  
We clean them using a custom `strip_html()` function that:
- removes all HTML tags  
- converts `<br>` into newlines  
- trims whitespace  

### **✔ Input Construction**
Each sample combines **questionTitle** and **questionText** into a single “input”.

In [1]:
def strip_html(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = re.sub(r"<br\s*/?>", "\n", s, flags=re.I)
    s = re.sub(r"<[^>]+>", "", s) 
    s = re.sub(r"\s+\n", "\n", s)
    return s.strip()

df = pd.read_csv(DATA_CSV_PATH)

candidate_cat_cols = [c for c in df.columns if c.lower() in {"questioncategory","category","tags","topics"}]
cat_col = candidate_cat_cols[0] if candidate_cat_cols else None

df["clean_answer"] = df["answerText"].astype(str).apply(strip_html)

def make_input(row):
    title = str(row.get("questionTitle","")).strip()
    body  = str(row.get("questionText","")).strip()
    if title and body and title not in body:
        return f"{title}\n\n{body}"
    return body or title

def make_instruction(row):
    base = "Provide an empathetic, evidence-based counseling reply."
    if cat_col and pd.notna(row.get(cat_col)) and str(row.get(cat_col)).strip():
        return f"{base} Topic: {str(row.get(cat_col)).strip()}."
    return base

usable = df[(df["clean_answer"].str.len() >= 20) & (df["questionText"].astype(str).str.len() >= 20)].copy()

records = []
for _, r in usable.iterrows():
    records.append({
        "instruction": make_instruction(r),
        "input": make_input(r),
        "output": r["clean_answer"]
    })

rng = np.random.RandomState(SEED)
idx = np.arange(len(records))
rng.shuffle(idx)
n = len(idx); n_train = int(0.8*n); n_dev = int(0.1*n)
train_idx = idx[:n_train]; dev_idx = idx[n_train:n_train+n_dev]; test_idx = idx[n_train+n_dev:]

splits = {
    "train": [records[i] for i in train_idx],
    "dev":   [records[i] for i in dev_idx],
    "test":  [records[i] for i in test_idx],
}

print({k: len(v) for k,v in splits.items()})

NameError: name 'pd' is not defined

We use **Qwen/Qwen2.5-0.5B-Instruct** — a ~0.5B parameter instruct model suitable for CPU demos.

- MAX_LEN capped at 1024
- Batch sizes are small (CPU-friendly)
- We will do baseline generation with the base model (no training)


In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"  
USE_4BIT = True
USE_BF16 = True  
MAX_LEN = 1024
BATCH_SIZE = 2
GRAD_ACCUM = 16
EPOCHS = 1     
LR = 2e-4
WARMUP_RATIO = 0.03

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
USE_4BIT = True         
USE_BF16 = True          

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

def load_base_model(model_name: str, use_4bit: bool, use_bf16: bool):
    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    bnb_cfg = None
    if use_4bit:
        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True
        )

    dtype_val = torch.bfloat16 if (use_bf16 and torch.cuda.is_available()) else None

    # FIX: dtype → torch_dtype
    def _create_model(_bnb_cfg):
        return AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=_bnb_cfg,
            torch_dtype=dtype_val   
        )

    try:
        base_model = _create_model(bnb_cfg)
    except Exception as e:
        print("[WARN] 4-bit load failed, falling back to non-quantized FP16/BF16. Reason:")
        print("       ", repr(e))
        base_model = _create_model(None)  
        use_4bit = False

    base_model.eval()
    return tokenizer, base_model, use_4bit

tokenizer, base_model, USE_4BIT = load_base_model(MODEL_NAME, USE_4BIT, USE_BF16)
print("Loaded model. 4-bit active:", USE_4BIT, "| CUDA available:", torch.cuda.is_available())


[WARN] 4-bit load failed, falling back to non-quantized FP16/BF16. Reason:
        AttributeError("'frozenset' object has no attribute 'discard'")
Loaded model. 4-bit active: False | CUDA available: False


## Prompt Formatting + Baseline Response Generation

This section defines **how prompts are constructed** and **how the baseline model generates answers** before any fine-tuning.

---

##  `format_prompt()`  
This function formats each example into an instruction-following prompt.

```python
def format_prompt(inst: str, inp: str) -> str:
    if inp and inp.strip():
        return f"Instruction: {inst}\nInput: {inp}\nAnswer:"
    return f"Instruction: {inst}\nAnswer:"


In [ ]:
def format_prompt(inst: str, inp: str) -> str:
    if inp and inp.strip():
        return f"Instruction: {inst}\nInput: {inp}\nAnswer:"
    return f"Instruction: {inst}\nAnswer:"

@torch.no_grad()
def generate_responses(model, examples, max_new_tokens=256, do_sample=False, temperature=0.7, top_p=0.9):
    outputs = []
    for ex in examples:
        prompt = format_prompt(ex["instruction"], ex["input"])
        enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_LEN).to(model.device)
        out = model.generate(**enc, max_new_tokens=max_new_tokens, do_sample=do_sample, temperature=temperature, top_p=top_p)
        text = tokenizer.decode(out[0], skip_special_tokens=True)
        pred = text[len(prompt):].strip()
        outputs.append(pred)
    return outputs

## Baseline Output Generation (Before Fine-Tuning)

This section generates the **baseline responses** from the untrained model (Qwen2.5-0.5B-Instruct).  
These outputs serve as the *“before fine-tuning”* reference point.

---

### 🔹 Move Model to CPU
```python
base_model.to("cpu")


In [ ]:
base_model.to("cpu")

TEST_SAMPLES = 10 #10 rows
test_subset = splits["test"][:TEST_SAMPLES]

baseline_outputs = generate_responses(base_model, test_subset, do_sample=False)

baseline_df = pd.DataFrame({
    "instruction": [ex["instruction"] for ex in test_subset],
    "input":       [ex["input"] for ex in test_subset],
    "reference":   [ex["output"] for ex in test_subset],
    "baseline":    baseline_outputs
})

baseline_csv = OUT_DIR / "baseline_outputs.csv"
baseline_df.to_csv(baseline_csv, index=False)
print("Saved baseline outputs to:", baseline_csv)
baseline_df.head(3)


c:\Users\User\miniconda3\envs\psyche-r1\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  if self.epsilon_cutoff is not None and self.epsilon_cutoff != 0.0:
c:\Users\User\miniconda3\envs\psyche-r1\lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  minor_issues["eta_cutoff"] = greedy_wrong_parameter_msg.format(
c:\Users\User\miniconda3\envs\psyche-r1\lib\site-packages\transformers\generation\configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should se

Saved baseline outputs to: runs\baseline_outputs.csv


,instruction,input,reference,baseline
0,"Provide an empathetic, evidence-based counseli...",Why do my boyfriend and I have such trouble co...,Try having a conversation with your boyfriend ...,Communication is key to any relationship. When...
1,"Provide an empathetic, evidence-based counseli...",My new husband constantly talks to himself\n\n...,Some people simply talk to themselves as a way...,It's important to recognize that communication...
2,"Provide an empathetic, evidence-based counseli...",How can I keep a long distance relationship go...,Hello. You are asking a very good question abo...,It is understandable that you are feeling anxi...


## Convert Records into HuggingFace Datasets + Tokenisation

After we create the cleaned `records` list and the train/dev/test splits, we now need to convert them into a format suitable for supervised fine-tuning (SFT).  
This step transforms human-readable text into **token IDs**, and prepares the dataset so the model can learn to predict counselor-style answers.

---

### 🔹 Convert Python Records → HuggingFace Dataset
We start by turning a normal list of Python dictionaries into a HuggingFace `Dataset` object:

```python
def build_hf_dataset(recs):
    return Dataset.from_list(recs)


In [ ]:
def build_hf_dataset(recs):
    return Dataset.from_list(recs)

def tokenize_sft(rec):
    inst, inp, out = rec["instruction"], rec["input"], rec["output"]
    prompt = format_prompt(inst, inp)
    full = prompt + " " + out + tokenizer.eos_token
    toks = tokenizer(full, max_length=MAX_LEN, truncation=True)
    toks["labels"] = toks["input_ids"].copy()
    return toks

train_ds = build_hf_dataset(splits["train"]).map(tokenize_sft, remove_columns=list(splits["train"][0].keys()))
dev_ds   = build_hf_dataset(splits["dev"]).map(tokenize_sft,   remove_columns=list(splits["dev"][0].keys()))
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

len(train_ds), len(dev_ds)

Map:   0%|          | 0/1102 [00:00<?, ? examples/s]

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

(1102, 137)

## Attach LoRA Adapters (CPU-safe) and Verify Trainable Parameters

This cell loads the base model **on CPU in FP32**, attaches **LoRA adapters** to attention/MLP blocks, switches the model to **train mode**, and verifies that only the LoRA layers are **trainable** (everything else stays frozen). This is what makes fine-tuning feasible on limited hardware.

---

### 🔹 1) Load the base model on CPU (float32)

We explicitly load the tokenizer and model on **CPU** with `torch_dtype=torch.float32`.  
FP32 on CPU ensures autograd works reliably without GPU/quantization dependencies.

Key points:
- If the tokenizer has no `pad_token`, we reuse `eos_token` to avoid padding errors.
- `model.eval()` here is just a safe default; we’ll flip to `train()` after adding LoRA.

What it enables:
- Deterministic, hardware-agnostic setup.
- No reliance on `bitsandbytes` or CUDA.

---

### 🔹 2) Pick LoRA target modules automatically

We scan `model.named_modules()` and match common projection names in **attention** (`q_proj`, `k_proj`, `v_proj`, `o_proj`) and **MLP** (`up_proj`, `down_proj`, `gate_proj`, or equivalents like `c_attn`, `c_proj`, `w1/w2/w3`).

Why:
- These layers control most of the model’s expressive power in language generation.
- LoRA on these layers gives strong adaptation with a tiny number of trainable parameters.

---

### 🔹 3) Define LoRA config and wrap the model

We use:
- `r=16`, `lora_alpha=32`, `lora_dropout=0.05`
- `bias="none"` (don’t touch original biases)
- `task_type="CAUSAL_LM"`

Effect:
- Only a **small adapter** is trained (~1.7% of total parameters), keeping the base weights frozen.
- Memory and compute are minimized — ideal for CPU fine-tuning.

---

### 🔹 4) Switch to train mode and confirm what’s trainable

- `model.train()` enables gradient flow through LoRA adapters.
- We list parameters with `requires_grad=True` to confirm LoRA attached correctly.
- Typical printout:
  - **Total params** (count of named tensors, not total scalar count)
  - **Trainable params** (only LoRA tensors)
  - First few trainable tensor names (e.g., `...q_proj.lora_A/B...`)

**What you should see**
- Dozens/hundreds of LoRA tensors listed as trainable.
- A summary like:  
  `trainable params: 8,798,208 || all params: 502,830,976 || trainable%: ~1.75%`


---

### ✅ Outcome of this cell

- Base model: **on CPU, FP32** (safe and portable).
- LoRA: **attached to attention/MLP** blocks.
- Training mode: **enabled**.
- Diagnostics: **printed** list of trainable LoRA parameters and a % summary.

This sets up the model for the **manual CPU training loop** in the next cell.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

FORCE_CPU = True
USE_BF16 = False       
USE_4BIT = False        

def load_base_model_cpu_fp32(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cpu",
        torch_dtype=torch.float32   
    )

    model.eval()
    return tokenizer, model

tokenizer, train_model = load_base_model_cpu_fp32(MODEL_NAME)

def guess_lora_targets(m):
    names = [n for n, _ in m.named_modules()]
    keys = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "up_proj", "down_proj", "gate_proj",
        "c_attn", "c_proj",
        "w1", "w2", "w3"
    ]
    return sorted({k for k in keys if any(k in n for n in names)})

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=guess_lora_targets(train_model),
)

train_model = get_peft_model(train_model, lora_cfg)

train_model.train()
train_model.to("cpu")

trainable_params = [(n, p) for n, p in train_model.named_parameters() if p.requires_grad]
frozen_params = [(n, p) for n, p in train_model.named_parameters() if not p.requires_grad]

print(f"Total params: {len(trainable_params) + len(frozen_params)}")
print(f"Trainable params: {len(trainable_params)}")

if not trainable_params:
    raise RuntimeError("❌ No trainable parameters — LoRA didn't attach.")

print("First few trainable params:")
for n, _ in trainable_params[:10]:
    print("  ", n)

train_model.print_trainable_parameters()
print("LoRA attached successfully. Device:", next(train_model.parameters()).device)


Total params: 626
Trainable params: 336
First few trainable params:
   base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
   base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
   base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
   base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
   base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
   base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
   base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
   base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
   base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight
   base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight
trainable params: 8,798,208 || all params: 502,830,976 || trainable%: 1.7497
LoRA attached successfully. Device: cpu


## Manual Fine-Tuning Loop on CPU (LoRA-only training)

This cell performs **supervised fine-tuning** on CPU using a **custom PyTorch loop**. We only train the **LoRA adapter weights** (the base model stays frozen), which makes training feasible without a GPU.

---

### Environment + Device
- We disable external loggers (`WANDB_*` env vars) and **hide CUDA** (`CUDA_VISIBLE_DEVICES=""`) to guarantee a **CPU-only** run, avoiding accidental GPU calls.
- `device = torch.device("cpu")` and `train_model.to(device)` move the LoRA-wrapped model to CPU.
- `train_model.train()` enables dropout and gradient computation.

---

### Hyperparameters
- `EPOCHS = 1` — one pass over the training set for a quick demo.
- `BATCH_SIZE = 1` — keeps memory usage tiny on CPU.
- `GRAD_ACCUM = 16` — **gradient accumulation** simulates an effective batch of ~16 examples without increasing RAM.
- `LR = 2e-4` — learning rate for AdamW.
- `MAX_STEPS = None` — optional early stop (keep `None` for full epoch).

---

### DataLoader
```python
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collator
)


In [ ]:
import os
import math
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"
os.environ["DISABLE_WANDB"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "" 

device = torch.device("cpu")
train_model.to(device)
train_model.train()

EPOCHS = 1
BATCH_SIZE = 1
GRAD_ACCUM = 16
LR = 2e-4
MAX_STEPS = None 

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collator
)

optim_params = [p for p in train_model.parameters() if p.requires_grad]
optimizer = AdamW(optim_params, lr=LR)

global_step = 0
optimizer.zero_grad()

print("Starting manual fine-tune on CPU:")
print(f"Trainable tensors: {len(optim_params)}")
print(f"Device: {device}")
print(f"Epochs: {EPOCHS}, Grad Accum: {GRAD_ACCUM}")

for epoch in range(EPOCHS):
    print(f"\n===== Epoch {epoch+1}/{EPOCHS} =====")
    for batch_idx, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = train_model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
        )
        loss = outputs.loss

        (loss / GRAD_ACCUM).backward()

        if (batch_idx + 1) % GRAD_ACCUM == 0:
            torch.nn.utils.clip_grad_norm_(optim_params, max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            print(f"[step {global_step}] loss={loss.item():.4f}")

            if MAX_STEPS is not None and global_step >= MAX_STEPS:
                break

    if MAX_STEPS is not None and global_step >= MAX_STEPS:
        break

if (batch_idx + 1) % GRAD_ACCUM != 0:
    torch.nn.utils.clip_grad_norm_(optim_params, max_norm=1.0)
    optimizer.step()
    optimizer.zero_grad()
    global_step += 1
    print(f"[final step {global_step}] (last partial batch)")

print("\nFinished CPU fine-tuning loop.")

adapter_out = OUT_DIR / "psyche_r1_sft"
adapter_out.mkdir(parents=True, exist_ok=True)
train_model.save_pretrained(str(adapter_out))
tokenizer.save_pretrained(str(adapter_out))
print(f"Saved fine-tuned LoRA adapter to: {adapter_out}")


Starting manual fine-tune on CPU:
Trainable tensors: 336
Device: cpu
Epochs: 1, Grad Accum: 16

===== Epoch 1/1 =====
[step 1] loss=2.6685
[step 2] loss=2.8328
[step 3] loss=2.4674
[step 4] loss=2.5675
[step 5] loss=2.6215
[step 6] loss=2.3738
[step 7] loss=2.6239
[step 8] loss=2.7623
[step 9] loss=2.5349
[step 10] loss=3.0732
[step 11] loss=2.4597
[step 12] loss=2.7410
[step 13] loss=2.0946
[step 14] loss=1.9922
[step 15] loss=1.9471
[step 16] loss=2.4989
[step 17] loss=2.6323
[step 18] loss=2.2356
[step 19] loss=2.1088
[step 20] loss=2.4865
[step 21] loss=2.2616
[step 22] loss=2.6919
[step 23] loss=2.2478
[step 24] loss=2.1239
[step 25] loss=2.4429
[step 26] loss=3.3551
[step 27] loss=2.0640
[step 28] loss=2.1978
[step 29] loss=2.3860
[step 30] loss=2.3989
[step 31] loss=2.0638
[step 32] loss=2.2060
[step 33] loss=2.5441
[step 34] loss=2.2393
[step 35] loss=3.0534
[step 36] loss=2.3217
[step 37] loss=2.7821
[step 38] loss=2.5646
[step 39] loss=2.2073
[step 40] loss=2.1424
[step 41] l

KeyboardInterrupt: 

## Reload Fine-Tuned Adapters + Compare Against Baseline

This cell **reloads the base model on CPU**, **attaches the saved LoRA adapters**, generates outputs on the same test subset, and writes a **side-by-side CSV** (`baseline_vs_finetuned.csv`) for inspection.


In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM
from peft import PeftModel

ft_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cpu",          
    torch_dtype=torch.float32  
)

ft_model = PeftModel.from_pretrained(ft_model, str(OUT_DIR / "psyche_r1_sft"))
ft_model.eval()
ft_model.to("cpu")

print("Fine-tuned LoRA adapter loaded and ready for evaluation")

ft_outputs = generate_responses(
    ft_model,
    test_subset,
    do_sample=False  
)

compare_df = pd.DataFrame({
    "instruction": [ex["instruction"] for ex in test_subset],
    "input":       [ex["input"] for ex in test_subset],
    "reference":   [ex["output"] for ex in test_subset],
    "baseline":    baseline_df["baseline"],
    "finetuned":   ft_outputs
})

cmp_csv = OUT_DIR / "baseline_vs_finetuned.csv"
compare_df.to_csv(cmp_csv, index=False)

print(" Saved comparison to:", cmp_csv)
display(compare_df.head(3))


ValueError: Can't find 'adapter_config.json' at 'runs\psyche_r1_sft'